In [1]:
import pandas as pd
import update_vars
import utils

import altair as alt
import datetime as dt

from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
gcsgp = GCSGeoPandas()

In [2]:
ridership = utils.read_format_ridership()

# Time Series Cost Info

* Revenue data sourced from Amtrak-provided Origin-Destination Ridership and Revenue'
* Cost data by route provided by CCJPA and SJJPA, costs for Route 99 reflect a sum of both JPA's portions

In [210]:
combined_costs = pd.read_excel(f"{update_vars.GCS_PATH}source_data/combined_costs.xlsx")

In [278]:
cost_route_to_route_short_name = {'1A': 'Route 1', '1B': 'Route 1', '1C': 'Route 1c', 99: 'Route 99',
                                 3: 'Route 3', 7: 'Route 7'}

In [279]:
combined_costs.columns = combined_costs.columns.map(lambda x: x.lower())
combined_costs = combined_costs.rename(columns={'month': 'trip_month', 'year': 'trip_year'})

In [280]:
combined_costs = combined_costs.assign(route_short_name = combined_costs.route.map(cost_route_to_route_short_name))

In [281]:
combined_costs = combined_costs.query('route != 99 | trip_month != 6') #  exclude 6/2025 since we only have ccjpa data

In [282]:
group_cols = ['route_short_name', 'trip_month', 'trip_year']

In [283]:
monthly_costs = combined_costs.groupby(group_cols)[['cost']].sum().reset_index()

In [284]:
# monthly = ridership.groupby(['ca_bus_route', 'trip_month', 'trip_year'])[['ridership', 'revenue']].sum().reset_index()
monthly = ridership.groupby(group_cols)[['ridership', 'revenue']].sum().reset_index()

In [285]:
monthly = monthly.merge(monthly_costs, on = group_cols)

In [286]:
date_from_row = lambda row: dt.date(year=row.trip_year, month=row.trip_month, day=1)

In [287]:
monthly = monthly.assign(date = monthly.apply(date_from_row, axis=1).astype('datetime64'))

In [288]:
shape_df = gcsgp.read_parquet(f'{update_vars.GCS_PATH}intermediate/sanj_shapes_trip_info_{update_vars.ANALYSIS_DATE}.parquet')

In [289]:
shape_df = shape_df[['route_short_name', 'route_long_name']].drop_duplicates()

In [290]:
monthly = monthly.merge(shape_df, on='route_short_name', how='left')

In [291]:
monthly = monthly.assign(net_cost = monthly.cost - monthly.revenue)

In [295]:
def trend_chart(df, col):
    
    title = col.replace('_', ' ').title()
    selection = alt.selection_point(fields=['route_short_name'], bind='legend')
    chart = (alt.Chart(monthly)
     .mark_line(point=True)
     .encode(
        x=alt.X('date:T', axis=alt.Axis(format="%Y %b")).title('Date'),
        y=alt.Y(f'{col}:Q').title(title),
        color=alt.Color('route_short_name:N', legend=alt.Legend(symbolLimit=0, labelFontSize=12, titleFontSize=14)),
        tooltip=['route_short_name', alt.Tooltip(f'{col}:Q', format=','), alt.Tooltip(f'{col}:Q', format=','), 'date',
                'route_long_name'],
        opacity=alt.when(selection).then(alt.value(1)).otherwise(alt.value(0.2))
        )
     .properties(width=800, height=400)
     .interactive()
    ).add_params(
        selection
    )
    chart = chart.configure_axis(labelFontSize=14, titleFontSize=16)
    return chart

## Net Cost Trend by route

* can mouse over points for additional info, scroll, and zoom chart
* shift-click routes in the legend to select or deselect for highlight

In [296]:
trend_chart(monthly, 'net_cost')

alt.Chart(...)

## Cost Trend by Route

In [298]:
trend_chart(monthly, 'cost')

alt.Chart(...)

## Revenue Trend by Route

In [297]:
trend_chart(monthly, 'revenue')

alt.Chart(...)